In [0]:
import os
base_folder = '##PUT YOUR FOLDER HERE##'

In [0]:
'''
Numpy is a Python numerical computation library that efficiently stores and processes numbers.
By numbers, we don't just mean scalars. Vectors, matrices and even higher order tensors are included as well.
Install: https://pypi.org/project/numpy/
'''
# First, import the library like so
import numpy as np
# Numpy makes the processing of data easy, it is one of the most useful things to learn.

In [0]:
# Create a vector by feeding a Python list into np.array()
v = np.array( [1,2,3] )
print(v)

In [0]:
# To find out the shape of the structure, numpy objects have a .shape attribute
print('The shape of v is: ', v.shape)

In [0]:
'''
Create a 3x3 matrix that looks like:
[1 2 3]
[4 5 6]
[7 8 9]
'''
# This is done by feeding np.array() with a list of lists (notice they are all the same length, otherwise numpy will complain)
m = np.array( [[1,2,3],[4,5,6],[7,8,9]] )
print(m)

In [0]:
# Again, find out the shape using the attribute
print('The shape of m is: ', m.shape)

In [0]:
# Besides creating numpy objects, you can also do things to them.
# You can read the elements of the object with indices
print('The 2nd element of v is: ', v[1])
print('The (2,2)-nd element of m is: ', m[1,1])
print('The 2nd row of m is: ', m[1])

In [0]:
# You can add/subtract/multiply objects
print('v+1 adds 1 to all the elements of v, yielding: ', v+1)
print('v*2 multiplies all elements of v by two, yielding: ', v*2)
# Even matrix multiplication
print('The matrix multiplication of m and v is: ', m@v)

In [0]:
# There are many tools available in numpy for manipulation of data. 
# Check out the starter tutorial at: https://numpy.org/devdocs/user/absolute_beginners.html for a flavour.

In [0]:
# PIL (Python Imaging Library)
'''
- PIL allows us to easily perform higher order operations on images. 
  - rotations, translations, sharpening require few lines of code
  - numpy <-> PIL, easy!
    - numpy -> ML models
    - PIL -> people
- It is not the only image processing library out there. Other popular ones include:
  - opencv: https://opencv.org. Real-time applications.
  - scikit-image: https://scikit-image.org. Wide toolset, well-supported.
  - Use what works for you.
'''

In [0]:
# The PIL library contains many modules. Image is one of them, and is going to be very commonly used in our case.
from PIL import Image

In [0]:
# You can read an Image directly using a filepath

# Change this to fit your folder and image names
imgs_folder = os.path.join( base_folder, '##YOUR IMAGE FOLDER##' )
image_filepath = os.path.join( imgs_folder, '##YOUR IMAGE NAME##' )
img = Image.open(image_filepath)

In [0]:
# Let's take a look at the image
from IPython.display import display
# I'm running this code on an IPython notebook, so this is the way I can show the image.
# If you are writing code on a regular non-notebook Python environment, just use: img.show()
display(img)

In [0]:
# You can read the size of the Image
W,H = img.size
print('The width of the Image is: ', W)
print('The height of the Image is: ', H)

In [0]:
# An Image can be easily converted into a numpy array.
img_arr = np.array(img)
# The numpy array derived from the Image is a HxWx3 matrix, where H and W are the height and width in pixels
# and there are 3 channels for RGB
print('The shape of the numpy arr is: ', img_arr.shape)

In [0]:
# PIL: img.size gets you W,H
# numpy: img.shape gets you H,W,C
# CAUTION: Notice that the height and width are flipped between the PIL and numpy representations. Be careful of this.

In [0]:
# It is also simple to go from numpy -> PIL
back2pil = Image.fromarray(img_arr)
display(back2pil)

In [0]:
# To demonstrate how easy it is to use numpy and PIL
# Let's write a function to stitch the two input PIL Images side-by-side
def stitch_lr(pil_img1, pil_img2):
  # First, we should shrink the images
  W1,H1 = pil_img1.size
  W1,H1 = W1//2, H1//2
  W2,H2 = pil_img2.size
  W2,H2 = W2//2, H2//2

  # Since we are joining at the side, the heights have to match
  Hmax = max(H1,H2)
  pil_img1 = pil_img1.resize( (W1,Hmax) )
  pil_img2 = pil_img2.resize( (W2,Hmax) )
  # Next, convert to numpy arrays
  img_arr1 = np.array(pil_img1)
  img_arr2 = np.array(pil_img2)
  stitched = np.hstack( (img_arr1, img_arr2) )
  return Image.fromarray( stitched )

In [0]:
twins = stitch_lr( img, img )
display(twins)

In [0]:
# With PIL, image transformations are pretty easy.
img_lrflip = img.transpose(Image.FLIP_LEFT_RIGHT)
display( stitch_lr(img,img_lrflip) )

In [0]:
img_udflip = img.transpose(Image.FLIP_TOP_BOTTOM)
display( stitch_lr(img, img_udflip) )

In [0]:
W,H = img.size
img_cropped = img.crop( (W*0.2, H*0.2, W*0.8, H*0.8) )
display( stitch_lr(img, img_cropped) )

In [0]:
img_rotated = img.rotate(45)
display( stitch_lr(img,img_rotated) )

In [0]:
img_translated = img.rotate(0, translate=(50, -50))
display( stitch_lr(img,img_translated) )

In [0]:
from PIL import ImageEnhance
img_colorbalanced = ImageEnhance.Color(img).enhance(2.0)
display( stitch_lr(img,img_colorbalanced) )

In [0]:
img_contrasted = ImageEnhance.Contrast(img).enhance(2.0)
display( stitch_lr(img,img_contrasted) )

In [0]:
img_darkened = ImageEnhance.Brightness(img).enhance(0.2)
display( stitch_lr(img,img_darkened) )

In [0]:
img_sharpened = ImageEnhance.Sharpness(img).enhance(10.0)
display( stitch_lr(img,img_sharpened) )

In [0]:
'''
When we feed images into deep neural networks for training, it is useful to transform these images with the above 
image processing methods, in random combinations and varying intensities.
In all the above transformations, you can still tell that there is a cat in the picture. By feeding the network with all of these
"versions" of the cat, you are encouraging it to generalise - to figure out what exactly gives the picture it's cat-ness.
This technique is known as data augmentation.
'''
# Functions to wrap the above examples, and introduce some randomness to the image transformations
def horizontal_flip(img):
    return img.transpose(Image.FLIP_LEFT_RIGHT)

def vertical_flip(img):
    return img.transpose(Image.FLIP_TOP_BOTTOM)

def crop(img, range=[0.2,0.8]):
  W,H = img.size
  crop_xmin = round( np.random.uniform() * range[0] * W )
  crop_xmax = round( W * (range[1] + np.random.uniform() * (1-range[1])) )
  crop_ymin = round( np.random.uniform() * range[0] * H )
  crop_ymax = round( H * (range[1] + np.random.uniform() * (1-range[1])) )
  return img.crop( (crop_xmin, crop_ymin, crop_xmax, crop_ymax) )

def translate(img, max_translate=0.2):
  tx_pix = round( (1 if np.random.uniform() >= 0.5 else -1) * (W * np.random.uniform() * max_translate) )
  ty_pix = round( (1 if np.random.uniform() >= 0.5 else -1) * (H * np.random.uniform() * max_translate) )
  return img.rotate(0, translate=(tx_pix, ty_pix))

def rotate(img, max_rotate_deg=30):
  rot_deg = round( (1 if np.random.uniform() >= 0.5 else -1) * (max_rotate_deg * np.random.uniform()) )
  return img.rotate(rot_deg)

def colorbalance(img, color_factors=[0.2,2.0]):
  factor = color_factors[0] + np.random.uniform() * (color_factors[1] - color_factors[0])
  enhancer = ImageEnhance.Color(img)
  return enhancer.enhance(factor)

def contrast(img, contrast_factors=[0.2,2.0]):
  factor = contrast_factors[0] + np.random.uniform() * (contrast_factors[1] - contrast_factors[0])
  enhancer = ImageEnhance.Contrast(img)
  return enhancer.enhance(factor)

def brightness(img, brightness_factors=[0.2,2.0]):
  factor = brightness_factors[0] + np.random.uniform() * (brightness_factors[1] - brightness_factors[0])
  enhancer = ImageEnhance.Brightness(img)
  return enhancer.enhance(factor)

def sharpness(img, sharpness_factors=[0.2,2.0]):
  factor = sharpness_factors[0] + np.random.uniform() * (sharpness_factors[1] - sharpness_factors[0])
  enhancer = ImageEnhance.Sharpness(img)
  return enhancer.enhance(factor)

# This is the default augmentation scheme that we will use for each training image.
def augmentations(img, p={'fliplr':0.8, 'flipud':0.8, 'crop':0.8, 'translate':0.8, 'rotate':0.8, 'color':0.8, 'contrast':0.8, 'brightness':0.8, 'sharpness':0}):
  if p['color'] > np.random.uniform():
    img = colorbalance(img)
  if p['contrast'] > np.random.uniform():
    img = contrast(img)
  if p['brightness'] > np.random.uniform():
    img = brightness(img)
  if p['sharpness'] > np.random.uniform():
    img = sharpness(img)
  
  if p['fliplr'] > np.random.uniform():
    img = horizontal_flip(img)
  if p['flipud'] > np.random.uniform():
    img = vertical_flip(img)
  if p['crop'] > np.random.uniform():
    img = crop(img)
  if p['translate'] > np.random.uniform():
    img = translate(img)
  if p['rotate'] > np.random.uniform():
    img = rotate(img)
  return img
  

In [0]:
# Let's see a few random augmentation
for _ in range(5):
  aug_img = augmentations( img )
  display(aug_img)

In [0]:
# In summary...
# Numpy and PIL are among the easiest to use libraries for manipulation of numerical data and images
# It is easy to convert between numpy and PIL
# But feel free to explore other image processing libraries, such as scikit-image and opencv as well.
# - opencv: fast, realtime
# - scikit-image: feature rich toolset
